In [1]:
%matplotlib inline

from Models.functions.plot import ROC, plot_confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from Models.functions.datasets import getDatasets

from Models.functions.preprocessing import clean
import pandas as pd
from nltk.corpus import stopwords
import re, string
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#from Models.functions.plot import ROC, plot_confusion_matrix

In [3]:
from bs4 import BeautifulSoup

def labelEncoder(y):
    le = LabelEncoder()
    le.fit(y)

    return (le.transform(y), len(le.classes_), list(le.classes_))

from nltk.corpus import stopwords
import re, string
import numpy as np

In [4]:
task = 'age'
dataset_name = 'brblogset'
MAX_FEATURES = 5000

datasets = getDatasets(task,'df', dataset_name)
for i in datasets.iterrows():

    name = i[1]['dataset_name']
    label = task
    ds_path = i[1]['path']

    # load training and test dataframes
    training_path = ds_path + '/' + i[1]['training']        
    #test_path = ds_path + '/' + i[1]['test']      

    df_training = pd.read_csv(training_path)#, usecols=cols)        
    #df_test = pd.read_csv(test_path)#, usecols=cols)        

    df_training['text'] = df_training['text'].apply(clean)
    #df_test['text'] = df_test['text'].apply(clean)
    X = df_training['text'].values
    y, n_classes, classes_name = labelEncoder(df_training[label].values)


In [5]:
len(df_training)

2081

In [6]:
def max_length(lines):
    """
    Calculate the maximum document length
    """
    return max([len(s.split()) for s in lines])

In [7]:
max_length = max_length(X)

result = [len(x.split()) for x in X]
print('Text informations:')
print('max length: %i / min length: %i / mean length: %i ' % (np.max(result),
                                                                np.min(result),
                                                                np.mean(result)))

Text informations:
max length: 246111 / min length: 0 / mean length: 2837 


In [8]:
def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    
    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    vec = TfidfVectorizer(max_features=MAX_FEATURES).fit(corpus)
    # vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [9]:
print(df_training.groupby(task).size())
c = df_training.groupby(task).size().index

age
a10-25    526
a26-40    817
a40+      738
dtype: int64


In [10]:
for i in range(n_classes):
    c1 = df_training[df_training[task] == c[i]]
    print(classes_name[i])
    text = [line.replace("\n", "") for line in c1.text]
    common_words = get_top_n_words(text, 10)
    for word, freq in common_words:
        print(word, freq)
    print("--")

a10-25
pra 19.11713110329236
ser 18.26320598815446
dia 13.912443381687353
tudo 13.886424874445177
vida 13.751203620880371
bem 11.618026516751847
aqui 10.90112017452841
ainda 10.60086446021793
sobre 10.512026346315631
amor 10.446542626423561
--
a26-40
ser 26.773788014688567
dia 21.242673043981682
vida 18.896776750077915
sobre 17.20846259592052
pra 16.9670529032183
bem 16.385928137988184
se 16.381608526791
deus 16.28922200731576
tudo 15.435237009843586
ainda 14.985437523534223
--
a40+
ser 25.053980419007935
se 19.15082968186628
vida 18.809873674024143
dia 18.46133564329977
sobre 16.960065818193332
deus 16.936701915912735
anos 16.059508678059544
todos 14.901210320329193
tudo 14.060458791293323
the 13.60200562041761
--


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
vec = TfidfVectorizer(max_features=MAX_FEATURES).fit(X_train)
X_train = vec.transform(X_train)
X_test = vec.transform(X_test)

In [12]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=1000, penalty='l1', max_iter=500, multi_class='auto')
clf = clf.fit(X_train, y_train)

/home/rafael/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [13]:
X_train.shape, X_test.shape

((1664, 206465), (417, 206465))

In [ ]:
import cnn_model

# EMBEDDING
MAX_NUM_WORDS  = 50000 #15000
EMBEDDING_DIM  = 300
MAX_SEQ_LENGTH = X_train.shape[1]# or 3200 #200
USE_GLOVE      = False

# MODEL
FILTER_SIZES   = [3,4,5]
FEATURE_MAPS   = [10,10,10]
DROPOUT_RATE   = 0.5

# LEARNING
BATCH_SIZE     = 20
NB_EPOCHS      = 40
RUNS           = 5
VAL_SIZE       = 0.2

def create_model():
    
    model = cnn_model.build_cnn(
            embedding_layer=emb_layer,
            num_words=MAX_NUM_WORDS,
            embedding_dim=EMBEDDING_DIM,
            filter_sizes=FILTER_SIZES,
            feature_maps=FEATURE_MAPS,
            max_seq_length=MAX_SEQ_LENGTH,
            dropout_rate=DROPOUT_RATE
    )
    
    model.compile(
            loss='binary_crossentropy',
            optimizer=Adadelta(clipvalue=3),
            metrics=['accuracy']
    )
    return model

Using TensorFlow backend.


In [ ]:
#import cnn_model
#from sklearn.model_selection import StratifiedKFold
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier

histories = []
test_loss = []
test_accs = []

predicted_y = []
expected_y = []

#K = StratifiedKFold(n_splits=2)

X_train_, X_val, y_train_, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

emb_layer = None
#if USE_GLOVE:
    #emb_layer = create_glove_embeddings()


model = KerasClassifier(build_fn=create_model, 
                        epochs=NB_EPOCHS,
                        batch_size=BATCH_SIZE,
                        verbose=1,
                        validation_data=(X_val, y_val),
                        callbacks=[#ModelCheckpoint('model-%i.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min'),
                            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, min_lr=0.01),
                            EarlyStopping(monitor='val_loss', min_delta=0.1, patience=4, verbose=1)
                        ])

history = model.fit(X_train, y_train)

Creating CNN 0.0.1
#############################################
Embedding:    no pre-trained embedding
Vocabulary size: 50000
Embedding dim: 300
Filter sizes: [3, 4, 5]
Feature maps: [10, 10, 10]
Max sequence: 206465
#############################################
Train on 1664 samples, validate on 333 samples
Epoch 1/40


# Evaluation

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=classes_name))
print()
cm = confusion_matrix(y_test, y_pred)
print(cm)
plot_confusion_matrix(cm, classes_name)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=classes_name))
print()
cm = confusion_matrix(y_test, y_pred)
print(cm)
plot_confusion_matrix(cm, classes_name)
y_score = model.predict_proba(X_test)
# ROC(y_test, y_score, n_classes, None, None, classes_name)